In [1]:
%matplotlib widget
%config InlineBackend.figure_format = "svg"

import addict
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from tqdm.notebook import tqdm
from typing import List, Dict, Tuple

import celeri
celeri = reload(celeri)

plt.rcParams["text.usetex"] = False # Plotting the global model is much much faster with tex fonts turned off

# Read in data files, create storage dictionaries, and do basic processing

In [2]:
command_file_name = "./data/western_north_america/basic_command.json"
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure, block = celeri.assign_block_labels(segment, station, block, mogi, sar)
assembly = addict.Dict()
operators = addict.Dict()
operators.meshes = [addict.Dict()] * len(meshes)
assembly = celeri.merge_geodetic_data(assembly, station, sar) # Not sure this works correctly

# Get elastic operators and TDE smoothing operators

In [3]:
# Get all elastic operators for segments and TDEs
celeri.get_elastic_operators(operators, meshes, segment, station, command)

# Get TDE smoothing operators
celeri.get_all_mesh_smoothing_matrices(meshes, operators)
celeri.get_all_mesh_smoothing_matrices_simple(meshes, operators)

# Calculate non-elastic operators

In [4]:
# Build all linear kinematic operators
operators.rotation_to_velocities = celeri.get_block_rotation_operator(station)
operators.global_float_block_rotation = celeri.get_global_float_block_rotation_operator(station)
assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)
operators.rotation_to_slip_rate = celeri.get_fault_slip_rate_partials(segment, block)
operators.block_strain_rate_to_velocities, strain_rate_block_idx = celeri.get_strain_rate_centroid_operator(block, station, segment)
operators.mogi_to_velocities = celeri.get_mogi_operator(mogi, station, command)

# Sketching out the assembly of the block model system

$$
\begin{bmatrix}
    \mathrm{geodetic \; velocities} \\
    \mathrm{plate \; rotation \; constraints} \\
    \mathrm{slip \; rate \; constraints} \\
    \mathrm{TDE \; rate \; constraints} \\
    \mathrm{TDE \; smoothing \; pseudodata = 0} \\
    \mathrm{InSAR \; LOS \; changes} 
\end{bmatrix}
=
\begin{bmatrix}
    \mathrm{(rotations-elastic \; segments) \; to \; velocities} & \mathrm{TDEs \; to \; velocities} & \mathrm{block \; strain \; rate \; to \; velocities} & \mathrm{Mogi \; to \; velocities}\\
    \mathrm{identities}                                          & 0                                   & 0                                & 0\\
    \mathrm{plate \; rotations \; to \; slip \; rates}           & 0                                   & 0                                & 0\\
    0                                                            & \mathrm{identities}                 & 0                                & 0\\
    0                                                            & \mathrm{TDE \; smoothing \; matrix} & 0                                & 0\\
    \mathrm{(rotations-elastic \; segments) \; to \; LOS}        & \mathrm{TDEs \; to \; LOS}          & \mathrm{block \; strain \; rate \; to \; velocities}  & \mathrm{Mogi \; to \; LOS}
\end{bmatrix}
\begin{bmatrix}
    \mathrm{plate \; rotation \; rates} \\
    \mathrm{TDE \; slip \; rates} \\
    \mathrm{block \; strain \; rates} \\
    \mathrm{Mogi \; sources} 
\end{bmatrix}
$$

# Block model with block rotations, fully locked segments and partially locked subduction zone using the full tde_matrix and smoothing matrix

In [5]:
# TODO: What should this converstion be?
DEG_PER_MYR_TO_RAD_PER_YR = np.deg2rad(1) / 1e6
DEG_PER_MYR_TO_RAD_PER_YR = 1 / 1e6


# Create dictionary to store indices and sizes for operator building
# idx.meshes[i].n_tri ???
idx = addict.Dict()
idx.n_tde = meshes[0].n_tde
idx.n_stations = assembly.data.n_stations
idx.vertical_velocities = np.arange(2, 3 * idx.n_stations, 3)
idx.n_blocks = len(block)
idx.n_block_constraints = assembly.data.n_block_constraints
idx.start_station_row = 0
idx.end_station_row = 2 * len(station)
idx.start_block_col = 0
idx.end_block_col = len(block)
idx.start_tde_col = idx.end_block_col
idx.end_tde_col = idx.start_tde_col + idx.n_tde
idx.meshes[0].start_tde_col = 0
idx.meshes[0].end_tde_col = 0
idx.meshes[0].start_tde_eigs_col = 0
idx.meshes[0].end_tde_eig_col = 0

# Initialize and build data vector
data_vector = np.zeros(2 * idx.n_stations + 2 * idx.n_tde + 3 * idx.n_block_constraints)
data_vector[0:2 * idx.n_stations] = celeri.interleave2(assembly.data.east_vel, assembly.data.north_vel) # add GPS data
data_vector[2 * idx.n_stations + 2 * idx.n_tde:2 * idx.n_stations + 2 * idx.n_tde + 3 * idx.n_block_constraints] = DEG_PER_MYR_TO_RAD_PER_YR * assembly.data.block_constraints # add plate motion constraints Not sure if units are correct

# Initialize linear operator
estimation = addict.Dict()
estimation.operator = np.zeros((2 * idx.n_stations + 2 * idx.n_tde + 3 * idx.n_block_constraints,
                     3 * idx.n_blocks + 2 * idx.n_tde))

# Insert block rotation and fully coupled segments
operators.rotation_to_slip_rate_to_okada_to_velocities = operators.slip_rate_to_okada_to_velocities @ operators.rotation_to_slip_rate
operators.rotation_to_slip_rate_to_okada_to_velocities = np.delete(operators.rotation_to_slip_rate_to_okada_to_velocities, idx.vertical_velocities, axis=0)
operators.rotation_to_velocities = np.delete(operators.rotation_to_velocities, idx.vertical_velocities, axis=0)

# Insert block rotations and elastic velocities from fully locked segments
estimation.operator[0:2 * idx.n_stations, 0:3 * idx.n_blocks] = operators.rotation_to_velocities - operators.rotation_to_slip_rate_to_okada_to_velocities

# Insert block motion constraints
estimation.operator[2 * idx.n_stations + 2 * idx.n_tde:2 * idx.n_stations + 2 * idx.n_tde + 3 * idx.n_block_constraints,
         0:3 * idx.n_blocks] = operators.block_motion_constraints

tde_keep_row_idx = celeri.get_keep_idx_12(operators.meshes[0].tde_to_velocities.shape[0])
tde_keep_col_idx = celeri.get_keep_idx_12(operators.meshes[0].tde_to_velocities.shape[1])
estimation.operator[0:2 * idx.n_stations,
         3 * idx.n_blocks : 3 * idx.n_blocks + 2 * idx.n_tde] = operators.meshes[0].tde_to_velocities[tde_keep_row_idx, :][:, tde_keep_col_idx]

# Insert TDE smoothing matrix
smoothing_keep_idx = celeri.get_keep_idx_12(operators.meshes[0].smoothing_matrix.shape[0])
estimation.operator[(2 * idx.n_stations + 3 * idx.n_block_constraints): (2 * idx.n_stations + 3 * idx.n_block_constraints + 2 * idx.n_tde),
         3 * idx.n_blocks : 3 * idx.n_blocks + 2 * idx.n_tde] = meshes[0].smoothing_weight * operators.meshes[0].smoothing_matrix.toarray()[smoothing_keep_idx, :][:, smoothing_keep_idx]

# Solve the overdetermined linear system
estimation.state_covariance_matrix = np.linalg.inv(estimation.operator.T @ estimation.operator)
estimation.state_vector = estimation.state_covariance_matrix @ estimation.operator.T @ data_vector

In [6]:
def post_process_estimation(estimation: Dict, station: pd.DataFrame, idx: Dict) -> None:
    """Calculate derived values derived from the block model linear estimate (e.g., velocities, undertainties)

    Args:
        estimation (Dict): Estimated state vector and model covariance
        station (pd.DataFrame): GPS station data
        idx (Dict): Indices and counts of data and array sizes
    """    
    
    estimation.predictions = estimation.operator @ estimation.state_vector
    estimation.vel = estimation.predictions[0:2 * idx.n_stations]
    estimation.east_vel = estimation.vel[0::2]
    estimation.north_vel = estimation.vel[1::2]

    # Estimate slip rate uncertainties
    estimation.slip_rate_sigma = np.sqrt(np.diag(operators.rotation_to_slip_rate @ estimation.state_covariance_matrix[0:3 * idx.n_blocks, 0:3 * idx.n_blocks] @ operators.rotation_to_slip_rate.T)) # I don't think this is correct because for the case when there is a rotation vector a priori
    estimation.strike_slip_rate_sigma = estimation.slip_rate_sigma[0::3]
    estimation.dip_slip_rate_sigma = estimation.slip_rate_sigma[1::3]
    estimation.tensile_slip_rate_sigma = estimation.slip_rate_sigma[2::3]

    # Calculate mean squared residual velocity
    estimation.east_vel_residual = estimation.east_vel - station.east_vel
    estimation.north_vel_residual = estimation.north_vel - station.north_vel

    # Extract TDE slip rates from state vector
    estimation.tde_rates = estimation.state_vector[3 * idx.n_blocks : 3 * idx.n_blocks + 2 * idx.n_tde]
    estimation.tde_strike_slip_rates = estimation.tde_rates[0::2]
    estimation.tde_dip_slip_rates = estimation.tde_rates[1::2]

    # Extract segment slip rates from state vector
    estimation.slip_rates = operators.rotation_to_slip_rate @ estimation.state_vector[0:3 * idx.n_blocks]
    estimation.strike_slip_rates = estimation.slip_rates[0::3]
    estimation.dip_slip_rates = estimation.slip_rates[1::3]
    estimation.tensile_slip_rates = estimation.slip_rates[2::3]

    # Calculate rotation only velocities
    estimation.vel_rotation = operators.rotation_to_velocities @ estimation.state_vector[0:3 * idx.n_blocks]
    estimation.east_vel_rotation = estimation.vel_rotation[0::2]
    estimation.north_vel_rotation = estimation.vel_rotation[1::2]

    # Calculate fully locked segment velocities
    estimation.vel_elastic_segment = operators.rotation_to_slip_rate_to_okada_to_velocities @ estimation.state_vector[0:3 * idx.n_blocks]
    estimation.east_vel_elastic_segment = estimation.vel_elastic_segment[0::2]
    estimation.north_vel_elastic_segment = estimation.vel_elastic_segment[1::2]

    # Calculate TDE velocities
    tde_keep_row_idx = celeri.get_keep_idx_12(operators.meshes[0].tde_to_velocities.shape[0])
    tde_keep_col_idx = celeri.get_keep_idx_12(operators.meshes[0].tde_to_velocities.shape[1])
    estimation.vel_tde = operators.meshes[0].tde_to_velocities[tde_keep_row_idx, :][:, tde_keep_col_idx] @ estimation.state_vector[3 * idx.n_blocks:]
    estimation.east_vel_tde = estimation.vel_tde[0::2]
    estimation.north_vel_tde = estimation.vel_tde[1::2]

post_process_estimation(estimation, station, idx)

# Plot model summary 

In [7]:
celeri.plot_estimation_summary(segment, station, estimation, lon_range=(225, 250), lat_range=(30, 52), quiver_scale=1e2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
operator_eps = copy.deepcopy(operator) + 1e-10
plt.figure()
plt.imshow(np.log10(np.abs(operator_eps)), cmap="plasma")
plt.colorbar()
plt.show()

NameError: name 'operator' is not defined

In [ ]:
meshes[0].n_tde